In [0]:
from google.colab import drive
drive.mount('/content/gdrive/')

In [0]:
ls

In [0]:
import os
os.chdir("/content/gdrive/My Drive/facial_expression_data")

In [0]:
ls

In [0]:
!pip3 install torch torchvision

#Preprocess Data

In [0]:
import torch
a = torch.Tensor([1]).cuda()
print(a)
torch.cuda.is_available()

In [0]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import matplotlib.pyplot as plt
import numpy as np
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models

In [0]:
def load_split_train_test(data_dir):
    train_values = []
    train_labels = []
    val_values = []
    val_labels = []
    test_values = []
    test_labels = []
  
    with open(data_dir,'r') as csvin:
      data=csv.reader(csvin)
    
      for row in data:
#       training, val and test set are already indicated in the csv file by the last attribute
        if row[-1] == 'Training':
          value = row[1]
          temp_train_row = []
          for pixel_value in value.split(' '):
            temp_train_row.append(int(pixel_value))
          temp_train_row = np.array(temp_train_row)
          temp_train_row = np.repeat(temp_train_row, 3)
          temp_train_row = temp_train_row.reshape((48, 48, 3)).astype('uint8')
          train_values.append(temp_train_row)
          train_labels.append(row[0])
        if row[-1] == 'PrivateTest':
          value = row[1]
          temp_val_row = []
          for pixel_value in value.split(' '):
            temp_val_row.append(int(pixel_value))
          temp_val_row = np.array(temp_val_row)
          temp_val_row = np.repeat(temp_val_row, 3)
          temp_val_row = temp_val_row.reshape((48, 48, 3)).astype('uint8')
          val_values.append(temp_val_row)
          val_labels.append(row[0])
        if row[-1] == 'PublicTest':
          value = row[1]
          temp_test_row = []
          for pixel_value in value.split(' '):
            temp_test_row.append(int(pixel_value))
          temp_test_row = np.array(temp_test_row)
          temp_test_row = np.repeat(temp_test_row, 3)
          temp_test_row = temp_test_row.reshape((48, 48, 3)).astype('uint8')
          test_values.append(temp_test_row)
          test_labels.append(row[0])
        
    train_values = np.array(train_values, dtype='uint8')
    train_labels = np.array(train_labels, dtype='int')
    val_values = np.array(val_values, dtype='uint8')
    val_labels = np.array(val_labels, dtype='int')
    test_values = np.array(test_values, dtype='uint8')
    test_labels = np.array(test_labels, dtype='int')
    
    return train_values, train_labels, val_values, val_labels, test_values, test_labels

In [0]:
x = [1,2,3]
x = np.array(x)
x = np.repeat(x, 3)
x = np.reshape(x, (3, 3))
print(x)

In [0]:
train_values, train_labels, val_values, val_labels, test_values, test_labels = load_split_train_test('data/fer2013.csv')
print(train_values[0])

In [0]:
print(np.shape(train_values[0]))

In [0]:
train_img = Image.fromarray(train_values[0])
# train_img = transforms.Resize(224)
train_img = train_img.convert('RGB')
print(train_img)
train_img.save("sample2.png")

# Training

In [0]:
import pandas as pd
import csv
import numpy as np
import h5py
from PIL import Image
from sklearn.externals import joblib

In [0]:
class fer2013_dataset(torch.utils.data.Dataset):
  def __init__(self, transform=None, partition='train'):
    self.to_tensor = transforms.ToTensor()
    self.transform = transform
    self.partition = partition
    self.data_info = pd.read_csv('data/fer2013.csv', header=None)
    train_values, train_labels, val_values, val_labels, test_values, test_labels = load_split_train_test('data/fer2013.csv')
    self.train_data = train_values
    self.train_label = train_labels
    self.val_data = val_values
    self.val_labels = val_labels
    self.test_data = test_values
    self.test_labels = test_labels
    
  def __getitem__(self, index):
    if self.partition == 'train':
      train_instance = self.train_data[index]
      train_img = Image.fromarray(train_instance)
      train_img = train_img.convert('RGB')
      if self.transform is not None:
        train_tensor = self.transform(train_img)
      return (train_tensor, self.train_label[index])
      
    if self.partition == 'val':
      val_instance = self.val_data[index]
      val_img = Image.fromarray(val_instance)
      val_img = val_img.convert('RGB')
      if self.transform is not None:
        val_tensor = self.transform(val_img)
      return (val_tensor, self.val_labels[index])
    
    if self.partition == 'test':
      test_instance = self.test_data[index]
      test_img = Image.fromarray(test_instance)
      test_img = test_img.convert('RGB')
      if self.transform is not None:
        test_tensor = self.transform(test_img)
      return (test_tensor, self.test_labels[index])
    
  def __len__(self):
    if self.partition == 'train':
      return len(self.train_label)
    if self.partition == 'val':
      return len(self.val_labels)
    if self.partition == 'test':
      return len(self.test_labels)

In [0]:
transformations = transforms.Compose([transforms.Resize(256),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),])

dataset = fer2013_dataset(transform=transformations, partition='train')
dataset_loader = torch.utils.data.DataLoader(dataset,
                                                    batch_size=64,
                                                    shuffle=True, num_workers=4)


In [0]:
val = fer2013_dataset(transform=transformations, partition='val')
val_loader = torch.utils.data.DataLoader(val,
                                                    batch_size=64,
                                                    shuffle=False, num_workers=4)

In [0]:
test = fer2013_dataset(transform=transformations, partition='test')
test_loader = torch.utils.data.DataLoader(test,
                                                    batch_size=64,
                                                    shuffle=False, num_workers=4)

In [0]:
for batch_idx, (images, labels) in enumerate(test_loader):
  print(labels)

In [0]:
class PreTrainedResNet(nn.Module):
  def __init__(self, num_classes, feature_extracting):
    super(PreTrainedResNet, self).__init__()
    
    #TODO1: Load pre-trained ResNet Model
    self.resnet18 = models.resnet18(pretrained=True)

    #Set gradients to false
    if feature_extracting:
      for param in self.resnet18.parameters():
          param.requires_grad = False
    
    #Replace last fc layer
    num_feats = self.resnet18.fc.in_features
    
    #TODO2: Replace fc layer in resnet to a linear layer of size (num_feats, num_classes)
    self.resnet18.fc = torch.nn.Linear(num_feats, num_classes)
    
  def forward(self, x):
    #TODO3: Forward pass x through the model
    x = self.resnet18(x)
    return x

In [0]:
def train(model, optimizer, criterion, epoch, num_epochs):
  model.train()
  epoch_loss = 0.0
  epoch_acc = 0.0
  
  for batch_idx, (images, labels) in enumerate(dataset_loader):
    #zero the parameter gradients
    optimizer.zero_grad()
    
    #move to GPU
    images, labels = images.cuda(), labels.cuda()
    
    #forward
    outputs = model.forward(images)
#     print(labels)
    
    loss = criterion(outputs, labels)
#     print("loss: " + str(loss))
    
    _, preds = torch.max(outputs.data, 1)
    
    loss.backward()
    optimizer.step()
    
    epoch_loss += loss.item()
    epoch_acc += torch.sum(preds == labels).item()
    
  epoch_loss /= 28709
  epoch_acc /= 28709
  
  print('TRAINING Epoch %d/%d Loss %.4f Accuracy %.4f' % (epoch, num_epochs, epoch_loss, epoch_acc))
  
  return epoch_loss, epoch_acc

# for i, (images, labels) in enumerate(dataset_loader):
#   print(len(images))

In [0]:
NUM_EPOCHS = 30
LEARNING_RATE = 0.01
BATCH_SIZE = 64
RESNET_LAST_ONLY = False #Fine tunes only the last layer. Set to False to fine tune entire network

#Initialize the model
model = PreTrainedResNet(7, RESNET_LAST_ONLY)
model = model.cuda()
joblib.dump(model, 'model.pkl')

#Setting the optimizer and loss criterion
optimizer = optim.SGD(model.parameters(), lr=LEARNING_RATE, momentum=0.9)
criterion = nn.CrossEntropyLoss()

plt.ioff()
fig = plt.figure()
loss_over_epochs = []
accuracy_over_epochs = []

#Begin Train
for epoch in range(NUM_EPOCHS):
  epoch_loss, epoch_acc = train(model, optimizer, criterion, epoch+1, NUM_EPOCHS)
  loss_over_epochs.append(epoch_loss)
  accuracy_over_epochs.append(epoch_acc)
  
joblib.dump(model, 'model.pkl')

plt.subplot(2, 1, 1)
plt.ylabel('Train loss')
plt.plot(np.arange(NUM_EPOCHS), loss_over_epochs, 'k-')
plt.title('train loss and accuracy')
plt.xticks(np.arange(NUM_EPOCHS, dtype=int))
plt.grid(True)

plt.subplot(2, 1, 2)
plt.plot(np.arange(NUM_EPOCHS), accuracy_over_epochs, 'b-')
plt.ylabel('Train accuracy')
plt.xlabel('Epochs')
plt.xticks(np.arange(NUM_EPOCHS, dtype=int))
plt.grid(True)
plt.savefig("plot.png")
plt.close(fig)
  
print("Finished Training")
print("-"*10)

#Test and evaluation of model

In [0]:
def test(model, criterion, repeats=2):
  model.eval()
  
  test_loss = 0.0
  test_acc = 0.0
  
  label = []
  predictions = []
  
  with torch.no_grad():
    for itr in range(repeats):
      for batch_idx, (images, labels) in enumerate(val_loader):
#         print(labels)
        label1 = labels.cpu()
        #move to GPU
        images, labels = images.cuda(), labels.cuda()

        #forward
        outputs = model.forward(images)

        loss = criterion(outputs, labels)

        _, preds = torch.max(outputs.data, 1)
        pred1 = preds.cpu()
        if(itr == 0):
          predictions.extend(np.array(pred1))
          label.extend(np.array(label1))

        test_loss += loss.item()
        test_acc += torch.sum(preds == labels).item()

    test_loss /= (3589*repeats)
    test_acc /= (3589*repeats)
    
    predictions = np.array(predictions)
    label = np.array(label)

    print('Test Loss: %.4f Test Accuracy %.4f' % (test_loss, test_acc))
    return predictions, label

In [0]:
predictions, label = test(model, criterion)
print(predictions, label)

In [0]:
from sklearn.metrics import confusion_matrix

In [0]:
print(confusion_matrix(predictions, label))

In [0]:
def test(model, criterion, repeats=2):
  model.eval()
  
  test_loss = 0.0
  test_acc = 0.0
  
  label = []
  predictions = []
  
  with torch.no_grad():
    for itr in range(repeats):
      for batch_idx, (images, labels) in enumerate(test_loader):
#         print(labels)
        label1 = labels.cpu()
        #move to GPU
        images, labels = images.cuda(), labels.cuda()

        #forward
        outputs = model.forward(images)

        loss = criterion(outputs, labels)

        _, preds = torch.max(outputs.data, 1)
        pred1 = preds.cpu()
        if(itr == 0):
          predictions.extend(np.array(pred1))
          label.extend(np.array(label1))

        test_loss += loss.item()
        test_acc += torch.sum(preds == labels).item()

    test_loss /= (3589*repeats)
    test_acc /= (3589*repeats)
    
    predictions = np.array(predictions)
    label = np.array(label)

    print('Test Loss: %.4f Test Accuracy %.4f' % (test_loss, test_acc))
    return predictions, label

In [0]:
predictions, label = test(model, criterion)

In [0]:
print(confusion_matrix(predictions, label))

#Test on Real Photo

In [0]:
!pip3 install face_recognition

In [0]:
import face_recognition
import argparse
import pickle
import cv2
import numpy as np
import scipy.misc

In [0]:
def detect(image):
    if image.shape[2] != 3:
        print('...')


    rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    boxes = face_recognition.face_locations(rgb,
    	model = 'cnn')
    print(boxes)
    images = []
    for box in boxes:
        img = image[box[0] : box[2] , box[3] : box[1]]
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        images.append(gray)
        
    for image in images:
      print(image)
      cv2_imshow(image)
        
    new_images = []
    for image in images:
      shape = np.shape(image)
      new_image = np.repeat(image, 3)
      new_image = new_image.reshape((shape[0], shape[1], 3)).astype('uint8')
      new_images.append(new_image)

    return new_images, boxes

In [0]:
test_img = cv2.imread('test5.jpg')
faces, boxes = np.array(detect(test_img))
print(np.shape(faces[0]))

In [0]:
class test_image(torch.utils.data.Dataset):
  def __init__(self, transform=None):
    self.to_tensor = transforms.ToTensor()
    self.transform = transform
    test_img = cv2.imread('test5.jpg')
    faces, boxes = np.array(detect(test_img))
    self.data = faces
    
  def __getitem__(self, index):
    instance = self.data[index]
    img = Image.fromarray(instance)
    img = img.convert('RGB')
    if self.transform is not None:
      instance_tensor = self.transform(img)
    return instance_tensor
    
  def __len__(self):
    return len(self.data)

In [0]:
transformations = transforms.Compose([transforms.Resize(256),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),])

dataset = test_image(transform=transformations)
dataset_loader = torch.utils.data.DataLoader(dataset,
                                                    batch_size=1,
                                                    shuffle=True, num_workers=4)

In [0]:
labels = []
with torch.no_grad():
  for batch_idx, images in enumerate(dataset_loader):
#   outputs = model.forward(images)
#   print(outputs)
    print(np.shape(np.array(images)))
    images = images.cuda()
    outputs = model.forward(images)
    _, preds = torch.max(outputs.data, 1)
    pred1 = preds.cpu()
    print(np.array(pred1))
    labels.extend(np.array(pred1))
print(labels)

In [0]:
labels = [5, 3, 3, 3, 3, 4, 3]

In [0]:
from google.colab.patches import cv2_imshow

In [0]:
class_list = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']
face_list = []
for ((top, right, bottom, left), label) in zip(boxes, labels):
  cv2.rectangle(test_img, (left, top), (right, bottom), (0, 255, 0), 2)
  y = top - 15 if top - 15 > 15 else top + 15
  cv2.putText(test_img, class_list[label], (left, y), cv2.FONT_HERSHEY_SIMPLEX,
            0.75, (0, 255, 0), 2)
  face_list.append((label, ((top+bottom)/2, (left+right)/2)))

print(face_list)
# cv2.namedWindow("output", cv2.WINDOW_NORMAL)
im2 = test_img.copy()
# im2[:,:,0] = test_img[:,:,2]
# im2[:, :, 2] = test_img[:, :, 0]
scipy.misc.imsave('kkk.jpg', imS)
cv2_imshow(im2)